In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression as OLS
import datetime,math
from random import *
from copy import *
seed(123)

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
##############################################################
##############################################################

In [4]:
def AddConstant(data_frame):
    data_frame = deepcopy(data_frame) 
    data_frame['const'] = np.ones(len(data_frame))
    cols = [data_frame.columns[-1]]+list(data_frame.columns[:-1])
    return pd.DataFrame(data_frame[cols]) 

In [5]:
##############################################################
##############################################################

In [6]:
fn = 'DATA/TestData.csv'
df = pd.read_csv(fn) 

In [7]:
df.head()

,DATE,SCORE,STU_ID,MCT,STU_IQ,STU_ST,STU_BF,ZIP
0,2017-09-10,59.4,GQCPF,0,94.0,3.0,0,70006
1,2016-03-29,75.2,NBXEV,0,87.0,13.0,0,71263
2,2016-11-03,110.4,EARNY,0,116.0,28.0,1,48223
3,2016-02-22,110.3,LXPHO,0,104.0,25.0,1,80524
4,2017-12-22,86.2,KGRBJ,1,118.0,0.0,1,70726


In [8]:
YVar  = 'SCORE' 
XVars = ['STU_IQ','STU_ST','STU_BF','MCT']

y = df[YVar]
X = df[XVars]
X = AddConstant(X) 

model = sm.OLS(y,X)
result = model.fit() 

In [9]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:              OLS              Adj. R-squared:     0.387       
Dependent Variable: SCORE            AIC:                1935078.0107
Date:               2018-10-16 14:42 BIC:                1935129.9013
No. Observations:   237546           Log-Likelihood:     -9.6753e+05 
Df Model:           4                F-statistic:        3.750e+04   
Df Residuals:       237541           Prob (F-statistic): 0.00        
R-squared:          0.387            Scale:              201.99      
-----------------------------------------------------------------------
              Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------
const        32.9525     0.2143   153.7886   0.0000   32.5325   33.3725
STU_IQ        0.5067     0.0020   258.6366   0.0000    0.5028    0.5105
STU_ST        0.4104     0.0017   244.0652   0.0000    0.4071    0.4137
STU_BF        5.6708     0.0602    94.1769   0.0000    5.5528    5.7888
MCT           7.8755     0.0752   104.7586   0.0000    7.7282    8.0229
---------------------------------------------------------------------
Omnibus:              4026.020       Durbin-Watson:          2.001   
Prob(Omnibus):        0.000          Jarque-Bera (JB):       4233.094
Skew:                 0.325          Prob(JB):               0.000   
Kurtosis:             3.077          Condition No.:          797     
=====================================================================

"""

In [10]:
Effect1 = result.params['MCT']
print 'Predicted MCT Effect on Test Score:',round(Effect1,3),'Points. :) Yayy!'

Predicted MCT Effect on Test Score: 7.876 Points. :) Yayy!


In [11]:
##############################################################
############################################################## 

In [12]:
# The bucketing strategy: 

In [13]:
G = df.groupby(['ZIP'])

In [14]:
IDs = list(G.sum().index)
mct_counts = np.array(G.sum()['MCT'])
tot_counts = np.array(G.count()['MCT']) 
non_counts = tot_counts-mct_counts

CountsDF = pd.DataFrame()
CountsDF['ID'] = IDs
CountsDF['TOT'] = tot_counts
CountsDF['NON'] = non_counts
CountsDF['MCT'] = mct_counts 
CountsDF['RAT'] = CountsDF['MCT']/CountsDF['TOT']
CountsDF['GEO'] = [round(a) for a in (non_counts*mct_counts)**0.5] 
CountsDF = CountsDF.sort_values('GEO',ascending=False)
CountsDF.index = list(range(len(CountsDF)))

In [15]:
CountsDF.head()

,ID,TOT,NON,MCT,RAT,GEO
0,90201,232,109,123,0.530172,116.0
1,92054,227,105,122,0.537445,113.0
2,92704,214,111,103,0.481308,107.0
3,90280,210,106,104,0.495238,105.0
4,90805,204,102,102,0.500000,102.0


In [16]:
CountsDF.tail()

,ID,TOT,NON,MCT,RAT,GEO
4721,31308,16,16,0,0.0,0.0
4722,31312,28,28,0,0.0,0.0
4723,31313,75,75,0,0.0,0.0
4724,31315,19,19,0,0.0,0.0
4725,10001,36,36,0,0.0,0.0


In [17]:
len(CountsDF)

4726

In [18]:
MinCount = 20
TF1 = CountsDF['NON'] >=MinCount
TF2 = CountsDF['MCT'] >=MinCount
CountsDF2 = pd.DataFrame(CountsDF[TF1&TF2]) 
GoodIDs = sorted(CountsDF2['ID'])

In [19]:
len(CountsDF2)

684

In [20]:
Rows = []
df2 = deepcopy(df)
for ID in GoodIDs: 
    df3 = df2[df2['ZIP']==ID] 
    df3.index = list(range(len(df3)))
    tf_mct = np.array(df3['MCT']==1) 
    ft_mct = np.logical_not(tf_mct) 
    mct_ave = df3[tf_mct]['SCORE'].mean()
    non_ave = df3[ft_mct]['SCORE'].mean() 
    Rows.append([ID, mct_ave, non_ave])
Rows.sort()
Cols = ['ZIP','MCT_SCORE','NON_SCORE'] 
Buckets = pd.DataFrame(Rows,columns=Cols) 
Buckets['MCT_DIF'] = Buckets['MCT_SCORE'] - Buckets['NON_SCORE'] 

In [21]:
Buckets.head()

,ZIP,MCT_SCORE,NON_SCORE,MCT_DIF
0,49686,91.335000,107.533333,-16.198333
1,50310,91.887500,97.550943,-5.663443
2,50613,91.630435,103.355072,-11.724638
3,51503,99.313333,105.732727,-6.419394
4,52761,111.869565,117.018868,-5.149303


In [22]:
AveBuckets = pd.DataFrame(Buckets.mean()).T
Effect2 = AveBuckets['MCT_DIF'][0] 
AveBuckets

,ZIP,MCT_SCORE,NON_SCORE,MCT_DIF
0,86556.44883,109.392201,115.972867,-6.580666


In [23]:
print 'Predicted MCT Effect on Test Score:',round(Effect2,3),'Points. :('

Predicted MCT Effect on Test Score: -6.581 Points. :(


In [24]:
##############################################################
##############################################################

In [25]:
df.head()  

,DATE,SCORE,STU_ID,MCT,STU_IQ,STU_ST,STU_BF,ZIP
0,2017-09-10,59.4,GQCPF,0,94.0,3.0,0,70006
1,2016-03-29,75.2,NBXEV,0,87.0,13.0,0,71263
2,2016-11-03,110.4,EARNY,0,116.0,28.0,1,48223
3,2016-02-22,110.3,LXPHO,0,104.0,25.0,1,80524
4,2017-12-22,86.2,KGRBJ,1,118.0,0.0,1,70726


In [26]:
FixedModel = "SCORE ~ STU_IQ+STU_ST+STU_BF+MCT"        #+DEC_YR" #+SEASON"
model = smf.mixedlm(FixedModel, df, groups=df["ZIP"])
result = model.fit() 

In [27]:
summary = result.summary()
summary

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
==========================================================
Model:            MixedLM Dependent Variable: SCORE       
No. Observations: 237546  Method:             REML        
No. Groups:       4726    Scale:              65.2039     
Min. group size:  6       Likelihood:         -844214.5192
Max. group size:  263     Converged:          Yes         
Mean group size:  50.3                                    
----------------------------------------------------------
              Coef.  Std.Err.    z     P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept     34.406    0.223  154.482 0.000 33.970 34.843
STU_IQ         0.504    0.001  448.298 0.000  0.502  0.506
STU_ST         0.410    0.001  424.715 0.000  0.408  0.412
STU_BF         7.482    0.035  215.649 0.000  7.414  7.550
MCT           -8.290    0.052 -160.531 0.000 -8.392 -8.189
Intercept RE 162.342    0.423                             
==========================================================

"""

In [28]:
Effect3 = result.params['MCT']
print 'Predicted MCT Effect on Test Score:',round(Effect3,3),'Points. 0_0 !!!'

Predicted MCT Effect on Test Score: -8.29 Points. 0_0 !!!


In [29]:
##############################################################
##############################################################

In [30]:
# [END] 